In [10]:
import argparse
from utils import *
from model import *
from train import *
import tqdm
from tqdm import tqdm
import wandb
import os


In [11]:
tokenizer = get_tokenizer('GPT2')

Adding pad_token to tokenizer



In [18]:
set_seed(42)

Random seed set as 42


In [19]:
class x:
    def __init__(self):
        return

args = x()
args.tokenizer_max_length = 64

In [20]:
use_pretrained_embeddings = True
freeze_pretrained_embeddings = True

Model1 = ConsolidatedModelClass(
        model_name='GPT2',
        num_layers=6,
        use_pretrained_embeddings=use_pretrained_embeddings,
        freeze_pretrained_embeddings=freeze_pretrained_embeddings,
        optimizer='AdamW',
        lr=0.001,
        tokenizer=tokenizer,
        scheduler=None,
        args=args,
        device='cuda')

In [21]:
use_pretrained_embeddings = True
freeze_pretrained_embeddings = False

Model2 = ConsolidatedModelClass(
        model_name='GPT2',
        num_layers=6,
        use_pretrained_embeddings=use_pretrained_embeddings,
        freeze_pretrained_embeddings=freeze_pretrained_embeddings,
        optimizer='AdamW',
        lr=0.001,
        tokenizer=tokenizer,
        scheduler=None,
        args=args,
        device='cuda')

In [22]:
use_pretrained_embeddings = False
freeze_pretrained_embeddings = False

Model3 = ConsolidatedModelClass(
        model_name='GPT2',
        num_layers=6,
        use_pretrained_embeddings=use_pretrained_embeddings,
        freeze_pretrained_embeddings=freeze_pretrained_embeddings,
        optimizer='AdamW',
        lr=0.001,
        tokenizer=tokenizer,
        scheduler=None,
        args=args,
        device='cuda')

In [23]:
dataset = load_hf_dataset('yelp_review_full')
dataset = prepare_datasets(dataset,0.01)   

print("Length of training dataset:", len(dataset['train']))
print("Length of validation dataset:", len(dataset['test']))

# ----------------------------- Load dataloaders ----------------------------- #
train_loader, val_loader, test_loader = prepare_dataloaders(dataset,batch_size=64,num_workers=16)

Found cached dataset yelp_review_full (/home/utsav/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/utsav/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-b0c3fdb0a0b3a7bb.arrow
Loading cached shuffled indices for dataset at /home/utsav/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-a0f1bfa2f136ab06.arrow


Converting dataset format to torch for split : train
Converting dataset format to torch for split : test
Length of training dataset: 6500
Length of validation dataset: 500


In [29]:
import gc
gc.collect()

4802

In [34]:
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   4121 MiB |  11791 MiB |  23891 GiB |  23887 GiB |\n|       from large pool |   4101 MiB |  11763 MiB |  23851 GiB |  23847 GiB |\n|       from small pool |     20 MiB |     33 MiB |     40 GiB |     40 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   4121 MiB |  11791 MiB |  23891 GiB |  23887 GiB |\n|       from large pool |   4101 MiB |  11763 MiB |

In [31]:
torch.cuda.mem_get_info()

(16443310080, 50962169856)

In [30]:
!nvidia-smi

Sat May  6 14:33:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:15:00.0 Off |                  Off |
| 33%   43C    P8    33W / 260W |  32919MiB / 48601MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     Off  | 00000000:2D:00.0 Off |                  Off |
| 34%   

In [26]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [24]:
for epoch in tqdm(range(200)):

    train_models(Model1,train_loader,args)
    evaluate_models(Model1,test_loader,args)

    train_models(Model2,train_loader,args)
    evaluate_models(Model2,test_loader,args)

    train_models(Model3,train_loader,args)
    evaluate_models(Model3,test_loader,args)

    print(Model1.losses['train_loss'][-1])
    print(Model1.losses['val_loss'][-1])

    print(Model2.losses['train_loss'][-1])
    print(Model2.losses['val_loss'][-1])

    print(Model3.losses['train_loss'][-1])
    print(Model3.losses['val_loss'][-1])

    # # for model in models:
    # train_loss = Model1.losses['train_loss'][-1]
    # val_loss = Model1.losses['val_loss'][-1]

    # print(f"\nhas train loss : {train_loss} \n \
    #                         and test loss : {val_loss}")

    # wandb.log({
    #     f"train loss" : train_loss,
    #     f"val loss" : val_loss,
    # },
    # step = epoch
    # )

102it [00:31,  3.25it/s]00:00<?, ?it/s]
100%|██████████| 8/8 [00:02<00:00,  3.73it/s]
102it [00:31,  3.28it/s]
100%|██████████| 8/8 [00:02<00:00,  3.87it/s]
102it [00:31,  3.29it/s]
  0%|          | 1/200 [01:42<5:40:35, 102.69s/it]

6.9978000977460075
6.233442842960358
7.21353894588994
6.429744899272919
6.517992230022655
6.181263744831085


102it [00:31,  3.28it/s]
100%|██████████| 8/8 [00:02<00:00,  3.90it/s]
18it [00:05,  3.16it/s]
  0%|          | 1/200 [02:23<7:55:41, 143.43s/it]


KeyboardInterrupt: 